# ARID1A Mutation Trans Effect on Phosphoproteomics

This notebook analyzes the trans effect of ARID1A mutation on interacting and other proteins Phosphoproteomics, in Endometrial, Colon, and Ovarian cancer.

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import collections
import re
import gseapy as gp
from gseapy.plot import barplot, dotplot
import matplotlib.pyplot as plt
import seaborn as sns

import cptac
import cptac.algorithms as al

en = cptac.Endometrial()
co = cptac.Colon()
ov = cptac.Ovarian()

### Select Gene

In [2]:
gene = "ARID1A"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
#omics = "transcriptomics"
omics = "phosphoproteomics"
#omics = "acetylproteomics"

# Interacting Proteins: Transcriptomics

## Endometrial

### Generate interacting protein list

In [4]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = en.get_phosphoproteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
SMARCC1
SUPT16H
CCND1
CREBBP
KAT2B
DNMT3A
SMARCA2
SMARCC2
KMT2D
ARID1A
SMARCE1
PHF10
BAZ1B
ARID1B
NF1
CDC5L
SMARCD2
SMARCA4
ACTL6A
BCL7B
SS18
DPF2
DPF3
ING1
PGR
BCL7C
HIC1


### Test for significant comparisons in any of interacting proteins

In [ ]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = en.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

Doing t-test comparisons



/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3157: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


                       Comparison       P_Value
0  ARID1A-S1755_phosphoproteomics  7.963747e-10
1  ARID1A-S1184_phosphoproteomics  4.338936e-07
2  ARID1A-S1604_phosphoproteomics  1.501846e-06
3   ARID1A-S363_phosphoproteomics  3.402219e-05





## Colon

### Generate interacting protein list

In [ ]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = co.get_phosphoproteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "_.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
SMARCC1
SUPT16H
SMARCD3
CREBBP
KAT2B
DNMT3A
SMARCA2
SMARCC2
KMT2D
ARID1A
PHF10
BAZ1B
ARID1B
NF1
CDC5L
SMARCD2
SMARCA4
ACTL6A
BCL7B
DPF2
ING1
PGR
BCL7C
HIC1


### Test for significant comparisons in any of interacting proteins

In [ ]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = co.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

SMARCC1 did not match any columns in phosphoproteomics dataframe. SMARCC1_phosphoproteomics column inserted, but filled with NaN.
SUPT16H did not match any columns in phosphoproteomics dataframe. SUPT16H_phosphoproteomics column inserted, but filled with NaN.
SMARCD3 did not match any columns in phosphoproteomics dataframe. SMARCD3_phosphoproteomics column inserted, but filled with NaN.
CREBBP did not match any columns in phosphoproteomics dataframe. CREBBP_phosphoproteomics column inserted, but filled with NaN.
KAT2B did not match any columns in phosphoproteomics dataframe. KAT2B_phosphoproteomics column inserted, but filled with NaN.
DNMT3A did not match any columns in phosphoproteomics dataframe. DNMT3A_phosphoproteomics column inserted, but filled with NaN.
SMARCA2 did not match any columns in phosphoproteomics dataframe. SMARCA2_phosphoproteomics column inserted, but filled with NaN.
SMARCC2 did not match any columns in phosphoproteomics dataframe. SMARCC2_phosphoproteomics column

## Ovarian

### Generate interacting protein list

In [ ]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = ov.get_phosphoproteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
SMARCC1
SUPT16H
CREBBP
SMARCB1
KAT2B
DNMT3A
SMARCA2
SMARCC2
KMT2D
ARID1A
SMARCE1
PHF10
BAZ1B
ARID1B
NF1
CDC5L
SMARCD2
SMARCA4
ACTL6A
BCL7B
DPF2
DPF3
ING1
PGR
BCL7C
HIC1


In [ ]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = ov.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df) 
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.loc[ind,'Label'] = 'Mutated'
    else:
        protdf.loc[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)
protdf = protdf.loc[:,~protdf.columns.duplicated()]

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

Doing t-test comparisons



/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3157: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


No significant comparisons.


# All Proteins: Phosphoproteomics

## Endometrial

In [ ]:
try:
    print("\nGene: ", gene)

    '''Use all proteins'''
    proteomics = en.get_proteomics()
    all_proteins = proteomics.columns
    #all_proteins = all_proteins[:100]

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = en.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=all_proteins)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the datafram correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")

except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  ARID1A
A1BG did not match any columns in phosphoproteomics dataframe. A1BG_phosphoproteomics column inserted, but filled with NaN.
A2M did not match any columns in phosphoproteomics dataframe. A2M_phosphoproteomics column inserted, but filled with NaN.
A2ML1 did not match any columns in phosphoproteomics dataframe. A2ML1_phosphoproteomics column inserted, but filled with NaN.
A4GALT did not match any columns in phosphoproteomics dataframe. A4GALT_phosphoproteomics column inserted, but filled with NaN.
AADAT did not match any columns in phosphoproteomics dataframe. AADAT_phosphoproteomics column inserted, but filled with NaN.
AAMP did not match any columns in phosphoproteomics dataframe. AAMP_phosphoproteomics column inserted, but filled with NaN.
AAR2 did not match any columns in phosphoproteomics dataframe. AAR2_phosphoproteomics column inserted, but filled with NaN.
AARS2 did not match any columns in phosphoproteomics dataframe. AARS2_phosphoproteomics column inserted, but fi

ACSF3 did not match any columns in phosphoproteomics dataframe. ACSF3_phosphoproteomics column inserted, but filled with NaN.
ACSL1 did not match any columns in phosphoproteomics dataframe. ACSL1_phosphoproteomics column inserted, but filled with NaN.
ACSM3 did not match any columns in phosphoproteomics dataframe. ACSM3_phosphoproteomics column inserted, but filled with NaN.
ACSS1 did not match any columns in phosphoproteomics dataframe. ACSS1_phosphoproteomics column inserted, but filled with NaN.
ACTA1 did not match any columns in phosphoproteomics dataframe. ACTA1_phosphoproteomics column inserted, but filled with NaN.
ACTA2 did not match any columns in phosphoproteomics dataframe. ACTA2_phosphoproteomics column inserted, but filled with NaN.
ACTB did not match any columns in phosphoproteomics dataframe. ACTB_phosphoproteomics column inserted, but filled with NaN.
ACTG1 did not match any columns in phosphoproteomics dataframe. ACTG1_phosphoproteomics column inserted, but filled with

AGPAT3 did not match any columns in phosphoproteomics dataframe. AGPAT3_phosphoproteomics column inserted, but filled with NaN.
AGPAT5 did not match any columns in phosphoproteomics dataframe. AGPAT5_phosphoproteomics column inserted, but filled with NaN.
AGR2 did not match any columns in phosphoproteomics dataframe. AGR2_phosphoproteomics column inserted, but filled with NaN.
AGR3 did not match any columns in phosphoproteomics dataframe. AGR3_phosphoproteomics column inserted, but filled with NaN.
AGT did not match any columns in phosphoproteomics dataframe. AGT_phosphoproteomics column inserted, but filled with NaN.
AHCY did not match any columns in phosphoproteomics dataframe. AHCY_phosphoproteomics column inserted, but filled with NaN.
AHSP did not match any columns in phosphoproteomics dataframe. AHSP_phosphoproteomics column inserted, but filled with NaN.
AIFM2 did not match any columns in phosphoproteomics dataframe. AIFM2_phosphoproteomics column inserted, but filled with NaN.


ANG did not match any columns in phosphoproteomics dataframe. ANG_phosphoproteomics column inserted, but filled with NaN.
ANGEL2 did not match any columns in phosphoproteomics dataframe. ANGEL2_phosphoproteomics column inserted, but filled with NaN.
ANGPT2 did not match any columns in phosphoproteomics dataframe. ANGPT2_phosphoproteomics column inserted, but filled with NaN.
ANGPT4 did not match any columns in phosphoproteomics dataframe. ANGPT4_phosphoproteomics column inserted, but filled with NaN.
ANGPTL1 did not match any columns in phosphoproteomics dataframe. ANGPTL1_phosphoproteomics column inserted, but filled with NaN.
ANGPTL4 did not match any columns in phosphoproteomics dataframe. ANGPTL4_phosphoproteomics column inserted, but filled with NaN.
ANKHD1 did not match any columns in phosphoproteomics dataframe. ANKHD1_phosphoproteomics column inserted, but filled with NaN.
ANKRD16 did not match any columns in phosphoproteomics dataframe. ANKRD16_phosphoproteomics column inserte

ARF1 did not match any columns in phosphoproteomics dataframe. ARF1_phosphoproteomics column inserted, but filled with NaN.
ARF3 did not match any columns in phosphoproteomics dataframe. ARF3_phosphoproteomics column inserted, but filled with NaN.
ARFRP1 did not match any columns in phosphoproteomics dataframe. ARFRP1_phosphoproteomics column inserted, but filled with NaN.
ARG2 did not match any columns in phosphoproteomics dataframe. ARG2_phosphoproteomics column inserted, but filled with NaN.
ARIH2 did not match any columns in phosphoproteomics dataframe. ARIH2_phosphoproteomics column inserted, but filled with NaN.
ARL1 did not match any columns in phosphoproteomics dataframe. ARL1_phosphoproteomics column inserted, but filled with NaN.
ARL15 did not match any columns in phosphoproteomics dataframe. ARL15_phosphoproteomics column inserted, but filled with NaN.
ARL4A did not match any columns in phosphoproteomics dataframe. ARL4A_phosphoproteomics column inserted, but filled with NaN

...

TOR2A did not match any columns in phosphoproteomics dataframe. TOR2A_phosphoproteomics column inserted, but filled with NaN.
TOR3A did not match any columns in phosphoproteomics dataframe. TOR3A_phosphoproteomics column inserted, but filled with NaN.
TP53I3 did not match any columns in phosphoproteomics dataframe. TP53I3_phosphoproteomics column inserted, but filled with NaN.
TPBG did not match any columns in phosphoproteomics dataframe. TPBG_phosphoproteomics column inserted, but filled with NaN.
TPGS2 did not match any columns in phosphoproteomics dataframe. TPGS2_phosphoproteomics column inserted, but filled with NaN.
TPH1 did not match any columns in phosphoproteomics dataframe. TPH1_phosphoproteomics column inserted, but filled with NaN.
TPK1 did not match any columns in phosphoproteomics dataframe. TPK1_phosphoproteomics column inserted, but filled with NaN.
TPM3 did not match any columns in phosphoproteomics dataframe. TPM3_phosphoproteomics column inserted, but filled with NaN

TSPAN9 did not match any columns in phosphoproteomics dataframe. TSPAN9_phosphoproteomics column inserted, but filled with NaN.
TSPO did not match any columns in phosphoproteomics dataframe. TSPO_phosphoproteomics column inserted, but filled with NaN.
TSPYL4 did not match any columns in phosphoproteomics dataframe. TSPYL4_phosphoproteomics column inserted, but filled with NaN.
TSPYL5 did not match any columns in phosphoproteomics dataframe. TSPYL5_phosphoproteomics column inserted, but filled with NaN.
TSTD1 did not match any columns in phosphoproteomics dataframe. TSTD1_phosphoproteomics column inserted, but filled with NaN.
TTC12 did not match any columns in phosphoproteomics dataframe. TTC12_phosphoproteomics column inserted, but filled with NaN.
TTC13 did not match any columns in phosphoproteomics dataframe. TTC13_phosphoproteomics column inserted, but filled with NaN.
TTC17 did not match any columns in phosphoproteomics dataframe. TTC17_phosphoproteomics column inserted, but fille

UBAC2 did not match any columns in phosphoproteomics dataframe. UBAC2_phosphoproteomics column inserted, but filled with NaN.
UBALD1 did not match any columns in phosphoproteomics dataframe. UBALD1_phosphoproteomics column inserted, but filled with NaN.
UBALD2 did not match any columns in phosphoproteomics dataframe. UBALD2_phosphoproteomics column inserted, but filled with NaN.
UBB did not match any columns in phosphoproteomics dataframe. UBB_phosphoproteomics column inserted, but filled with NaN.
UBE2A did not match any columns in phosphoproteomics dataframe. UBE2A_phosphoproteomics column inserted, but filled with NaN.
UBE2B did not match any columns in phosphoproteomics dataframe. UBE2B_phosphoproteomics column inserted, but filled with NaN.
UBE2C did not match any columns in phosphoproteomics dataframe. UBE2C_phosphoproteomics column inserted, but filled with NaN.
UBE2D1 did not match any columns in phosphoproteomics dataframe. UBE2D1_phosphoproteomics column inserted, but filled 

UTP4 did not match any columns in phosphoproteomics dataframe. UTP4_phosphoproteomics column inserted, but filled with NaN.
UTP6 did not match any columns in phosphoproteomics dataframe. UTP6_phosphoproteomics column inserted, but filled with NaN.
UXS1 did not match any columns in phosphoproteomics dataframe. UXS1_phosphoproteomics column inserted, but filled with NaN.
UXT did not match any columns in phosphoproteomics dataframe. UXT_phosphoproteomics column inserted, but filled with NaN.
VAMP2 did not match any columns in phosphoproteomics dataframe. VAMP2_phosphoproteomics column inserted, but filled with NaN.
VAMP7 did not match any columns in phosphoproteomics dataframe. VAMP7_phosphoproteomics column inserted, but filled with NaN.
VARS2 did not match any columns in phosphoproteomics dataframe. VARS2_phosphoproteomics column inserted, but filled with NaN.
VBP1 did not match any columns in phosphoproteomics dataframe. VBP1_phosphoproteomics column inserted, but filled with NaN.
VCAM

WNT7A did not match any columns in phosphoproteomics dataframe. WNT7A_phosphoproteomics column inserted, but filled with NaN.
WRB did not match any columns in phosphoproteomics dataframe. WRB_phosphoproteomics column inserted, but filled with NaN.
WRB-SH3BGR did not match any columns in phosphoproteomics dataframe. WRB-SH3BGR_phosphoproteomics column inserted, but filled with NaN.
XAGE1A did not match any columns in phosphoproteomics dataframe. XAGE1A_phosphoproteomics column inserted, but filled with NaN.
XCL1 did not match any columns in phosphoproteomics dataframe. XCL1_phosphoproteomics column inserted, but filled with NaN.
XDH did not match any columns in phosphoproteomics dataframe. XDH_phosphoproteomics column inserted, but filled with NaN.
XG did not match any columns in phosphoproteomics dataframe. XG_phosphoproteomics column inserted, but filled with NaN.
XPNPEP3 did not match any columns in phosphoproteomics dataframe. XPNPEP3_phosphoproteomics column inserted, but filled wi

## Colon

In [ ]:
try:
    print("\nGene: ", gene)

    '''Use all proteins'''
    proteomics = co.get_proteomics()
    all_proteins = proteomics.columns
    #all_proteins = all_proteins[:100]

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = co.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=all_proteins)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the datafram correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")

except Exception as e:
    print("Error in Comparison")
    print(e)

## Ovarian

In [ ]:

print("\nGene: ", gene)

'''Use all proteins'''
proteomics = ov.get_proteomics()
all_proteins = list(set(proteomics.columns))

'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = ov.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=all_proteins)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
protdf = protdf.loc[:,~protdf.columns.duplicated()]

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the datafram correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")


## Conclusions

There were no significantly altered phosphorylation sites in any cancer type when the MUC16 gene was mutated.